In [1]:
import numpy as np

In [2]:
"""
Suppose N vectors with M dimensions

M represents the number of factors, in our case the number of sectors that we decide to use

This gives us:

 > S, the NxM matrix of stocks split up into their factor exposures
 > E, the MxM covariance matrix across the different factors
 > I, the NxN diagonal matrix of idiosyncratic risks/variances
 > V, the Nx1 vector of idiosyncratic variances of the stocks

To calculate the best vector combination of S to get some target vector t

1) Find the vector s in S that is closest to pointing in the same direction as error e    [initially, e = t]
2) Calculate the new error e by taking e = e - e * s / (s * s)   [we update e by consider how much can be projected onto best vector s and subtracted away]

^ This ignores the problem on constraining on something like variance for now
"""

'\nSuppose N vectors with M dimensions\n\nM represents the number of factors, in our case the number of sectors that we decide to use\n\nThis gives us:\n\n > S, the NxM matrix of stocks split up into their factor exposures\n > E, the MxM covariance matrix across the different factors\n > I, the NxN diagonal matrix of idiosyncratic risks/variances\n > V, the Nx1 vector of idiosyncratic variances of the stocks\n\nTo calculate the best vector combination of S to get some target vector t\n\n1) Find the vector s in S that is closest to pointing in the same direction as error e    [initially, e = t]\n2) Calculate the new error e by taking e = e - e * s / (s * s)   [we update e by consider how much can be projected onto best vector s and subtracted away]\n\n^ This ignores the problem on constraining on something like variance for now\n'

In [3]:
N = 50
M = 5

S = np.random.rand(N, M) * 10 - 5
E = np.random.rand(M, N) / 50
E = E @ E.T
I = np.diag(np.random.rand(N))

t = np.random.rand(M) * 10 - 5

In [4]:
def optimize_vector_sum(vector_library, target, error_bound = 1e-5):
    error = target
    vector_sum = []

    vector_library_copy = vector_library.copy()

    while vector_library_copy.any() and np.linalg.norm(error) > error_bound:
        alignments = vector_library_copy @ error
        most_aligned_idx = np.argmax(abs(alignments))
        most_aligned_vector = vector_library[most_aligned_idx]

        scaling_coeff = np.dot(most_aligned_vector, error) / np.dot(most_aligned_vector, most_aligned_vector)
        vector_sum.append((most_aligned_idx, scaling_coeff))

        error = error - scaling_coeff * most_aligned_vector
        vector_library_copy[most_aligned_idx] = np.zeros_like(most_aligned_vector)

    final_vector = np.zeros_like(error)
    for vector_idx, coeff in vector_sum:
        final_vector += coeff * vector_library[vector_idx]

    return vector_sum, final_vector, error

In [5]:
print("Target:       ", t)
raw_sum, final_vec, error = optimize_vector_sum(S, t)
print("Construction: ", final_vec)

print("\n num vecs", len(raw_sum))

Target:        [ 2.2685471   1.32373219  3.06853891  4.56703558 -4.59342132]
Construction:  [ 2.26854737  1.32373062  3.0685419   4.56703278 -4.59341814]

 num vecs 19


In [6]:
"""
Now, to try and include the variance as a topic of optimization, we can define a loss function

Given the prior definitions, when we finally have weightings for each of our stocks, we can represent that as w, the vector of weightings

The risk associated with w can be represented as:

 > w * S * E * (w * S)T + w * I * w

In english, the square beta-wieghted sum of covariances plus the square weighted idiosyncratic risks

So, when selecting vectors with weights w1, w2, ..., wn, we can calculate the above and then normailize by the square of total weighting

To optimize on this, we can make a loss that is a combinated of the dot product for direction and the risk from above, then take the best option
"""

'\nNow, to try and include the variance as a topic of optimization, we can define a loss function\n\nGiven the prior definitions, when we finally have weightings for each of our stocks, we can represent that as w, the vector of weightings\n\nThe risk associated with w can be represented as:\n\n > w * S * E * (w * S)T + w * I * w\n\nIn english, the square beta-wieghted sum of covariances plus the square weighted idiosyncratic risks\n\nSo, when selecting vectors with weights w1, w2, ..., wn, we can calculate the above and then normailize by the square of total weighting\n\nTo optimize on this, we can make a loss that is a combinated of the dot product for direction and the risk from above, then take the best option\n'

In [ ]:
def optimize_constrainted_vector_sum(vector_library, target, covariance_matrix, idio_matrix, error_bound = 1e-5):
    error = target
    vector_sum = np.zeros(vector_library.shape[0])

    vector_library_copy = vector_library.copy()
    c = 0

    while vector_library_copy.any() and np.linalg.norm(error) > error_bound:
        all_coeffs = vector_library_copy @ error / (np.einsum("ij,ij->i", vector_library_copy, vector_library_copy) + 1e-8)
        possible_weights = np.array([vector_sum for i in all_coeffs]) + np.diag(all_coeffs)
        alignments = -abs(all_coeffs) / (error.T @ error) ** (1/2) * (np.einsum("ij,ij->i", vector_library_copy, vector_library_copy) + 1e-8) ** (1/2)
        variance = np.diag(possible_weights.T @ vector_library @ covariance_matrix @ (possible_weights.T @ vector_library).T + possible_weights.T @ idio_matrix @ possible_weights) / np.sum(abs(possible_weights), axis = 1)**2
        
        bias = .75 * np.exp(-c / len(vector_sum))
        loss = alignments * (1 - bias) + variance * bias

        best_idx = np.argmin(loss)

        best_vector = vector_library[best_idx]
        scaling_coeff = all_coeffs[best_idx]

        vector_sum[best_idx] += scaling_coeff
        error = error - scaling_coeff * best_vector

        c += 1

    final_vector = vector_sum.T @ vector_library

    return vector_sum / np.sum(vector_sum), final_vector, error

In [8]:
optimize_constrainted_vector_sum(S, t, E, I)

(array([-0.00000000e+00, -0.00000000e+00, -1.03419582e+02, -0.00000000e+00,
        -0.00000000e+00,  7.57174904e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  1.62495108e+02, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00,  4.95298376e-02,  4.90799579e+01,
        -2.20617324e-03, -0.00000000e+00,  1.40798807e+01, -0.00000000e+00,
         5.04904021e-01, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -1.31763038e-02,  8.04237835e-02, -0.00000000e+00, -0.00000000e+00,
         3.05442788e+00,  1.01342483e+00, -0.00000000e+00, -1.36710258e-01,
        -0.00000000e+00, -1.07329069e-02, -0.00000000e+00, -0.00000000e+00,
        -3.33519956e+01, -0.00000000e+00, -0.00000000e+00, -2.64814565e-04,
        -9.93768400e-01, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -9.69116280e-04, -0.00000000e+00]),
 array([ 2.26854486,  1.32372966,  3.0685415